### Objective:  Gather 20 years of NFL draft data and and build into usable relational database for analysis

Data Mining: Leverage pro-football-reference.com to collect NFL draft and combine data from 2004 through 2023.

Data Transformation: Consolidate data. Clean and standardized columns. 

Data Modeling: Create necessary dimenson tables and fact table with relational ID columns for normalization. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Create list of years we want data for
# Use the range function to potentially add more years later

year_list = [x for x in range(2004, 2024)]
print(year_list)

[2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


In [3]:
# Create list of URLs to scrape

url_list = []

for X in year_list:
    url = f"https://www.pro-football-reference.com/draft/{X}-combine.htm"
    url_list.append(url)

print(len(url_list))
url_list

20


['https://www.pro-football-reference.com/draft/2004-combine.htm',
 'https://www.pro-football-reference.com/draft/2005-combine.htm',
 'https://www.pro-football-reference.com/draft/2006-combine.htm',
 'https://www.pro-football-reference.com/draft/2007-combine.htm',
 'https://www.pro-football-reference.com/draft/2008-combine.htm',
 'https://www.pro-football-reference.com/draft/2009-combine.htm',
 'https://www.pro-football-reference.com/draft/2010-combine.htm',
 'https://www.pro-football-reference.com/draft/2011-combine.htm',
 'https://www.pro-football-reference.com/draft/2012-combine.htm',
 'https://www.pro-football-reference.com/draft/2013-combine.htm',
 'https://www.pro-football-reference.com/draft/2014-combine.htm',
 'https://www.pro-football-reference.com/draft/2015-combine.htm',
 'https://www.pro-football-reference.com/draft/2016-combine.htm',
 'https://www.pro-football-reference.com/draft/2017-combine.htm',
 'https://www.pro-football-reference.com/draft/2018-combine.htm',
 'https://

In [4]:
# Create empty df

col_list = ["Player", "Pos", "School", "College", "Ht", "Wt", "40yd", "Vertical", \
           "Bench", "Broad Jump", "3Cone", "Shuttle", "Drafted (tm/rnd/yr)"]

agg_df = pd.DataFrame(columns= col_list)

# Scrape URLs, add data to df

for url in url_list:
    data = pd.read_html(url)[0]
    agg_df = agg_df.append(data)
    

print(len(agg_df))
agg_df.head()

6841


,Player,Pos,School,College,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)
0,Derek Abney,WR,Kentucky,College Stats,5-9,179,4.58,33.5,NaN,120,6.89,4.08,Baltimore Ravens / 7th / 244th pick / 2004
1,Nathaniel Adibi,DE,Virginia Tech,College Stats,6-3,254,4.65,33.0,NaN,118,NaN,4.36,Pittsburgh Steelers / 5th / 145th pick / 2004
2,Roc Alexander,CB,Washington,College Stats,5-11,185,4.35,39.5,NaN,125,6.97,4.23,NaN
3,Rich Alexis,RB,Washington,College Stats,6-0,210,4.64,32.0,NaN,114,7.41,4.43,NaN
4,Jared Allen,DE,Idaho State,NaN,6-6,265,4.72,33.0,13,120,7.11,4.34,Kansas City Chiefs / 4th / 126th pick / 2004


In [6]:
# save raw data locally

#path = "/Users/Andrew/Desktop/Projects/NFL_draft/"

#pd.concat(df_list).to_csv(path + "raw_data/raw_data.csv", index=False)

In [7]:
# Drop College column, drop dups

agg_df = agg_df.drop(columns=["College"])
agg_df = agg_df.drop_duplicates(keep=False)

agg_df.head()

,Player,Pos,School,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)
0,Derek Abney,WR,Kentucky,5-9,179,4.58,33.5,NaN,120,6.89,4.08,Baltimore Ravens / 7th / 244th pick / 2004
1,Nathaniel Adibi,DE,Virginia Tech,6-3,254,4.65,33.0,NaN,118,NaN,4.36,Pittsburgh Steelers / 5th / 145th pick / 2004
2,Roc Alexander,CB,Washington,5-11,185,4.35,39.5,NaN,125,6.97,4.23,NaN
3,Rich Alexis,RB,Washington,6-0,210,4.64,32.0,NaN,114,7.41,4.43,NaN
4,Jared Allen,DE,Idaho State,6-6,265,4.72,33.0,13,120,7.11,4.34,Kansas City Chiefs / 4th / 126th pick / 2004


In [8]:
def split_column_fix_year(df):
    
# split "Drafted (tm/rnd/yr)" column into 4 columns
    df[["team", "round", "pick", "year"]] = df["Drafted (tm/rnd/yr)"].str.split("/", expand=True)
    
# add year value in year column for undrafted players
    df["year"] = df["year"].fillna(method='ffill')
    
    df = df.fillna("")

    return df

In [9]:
# Clean School column

def clean_school_column (df):   
    df = df.rename(columns={"School": "school_name"})
    df["school_name"] = df["school_name"].str.strip()
    df["school_name"] = df["school_name"].str.replace("State","St.") \
                                .str.replace("Ala-Birmingham","Alabama-Birmingham") \
                                .str.replace("Boston College","Boston Col.") \
                                .str.replace("West. Michigan","Western Michigan") \
                                .str.replace("Louisiana St","LSU") \
                                .str.replace("OH","Ohio") \
                                .str.replace("Middle Tenn. St.","Middle Tennessee St.") \
                                .str.replace("Tenn-Chattanooga","Tennessee-Chattanooga")
    return df

In [10]:
# Clean Team column

def clean_team_column (df):
    df = df.rename(columns={"team": "team_name"})
    df["team_name"] = df["team_name"].str.strip()
    replace_teams_dict = {"Oakland Raiders": "Las Vegas Raiders",
                         "Washington Redskins": "Washington Commanders",
                         "Washington Football Team": "Washington Commanders",
                         "St. Louis Rams": "Los Angeles Rams",
                         "San Diego Chargers": "Los Angeles Chargers"}
    df = df.replace({"team_name": replace_teams_dict})
    return df
    

In [11]:
# Clean Height column

def clean_height_column (df):
    df = df.rename(columns={"Ht": "height"})
    df["height"] = df["height"].str.strip()
    height_dict = {"5-4": "64", "5-5": "65", "5-6": "66", "5-7": "67", "5-8": "68",
                   "5-9": "69", "5-10": "70", "5-11": "71", "6-0": "72", "6-1": "73",
                   "6-2": "74", "6-3": "75", "6-4": "76",  "6-5": "77", "6-6": "78",
                   "6-7": "79", "6-8": "80", "6-9": "81", "6-10": "82", "6-11": "83"}
    df = df.replace({"height": height_dict})
    df["height"] = df["height"].replace("",0).astype(int)
    return df

In [12]:
# Clean Pick_Number column

def clean_pick_column (df):
    df = df.rename(columns={"Pick_Number": "pick"})
    df["pick"] = df["pick"].str.strip().str[:-7]
    df["pick"] = df["pick"].replace("",0).astype(int)
    return df

In [13]:
# Clean Round column

def clean_round_column (df):
    df = df.rename(columns={"Round": "round"})
    df["round"] = df["round"].str.strip().str[:-2]
    df["round"] = df["round"].replace("",0).astype(int)
    return df

In [14]:
#execute clean column functions

agg_df = split_column_fix_year(agg_df)
agg_df = clean_school_column(agg_df)
agg_df = clean_team_column(agg_df)
agg_df = clean_height_column(agg_df)
agg_df = clean_pick_column(agg_df)
agg_df = clean_round_column(agg_df)

In [15]:
agg_df.head()

,Player,Pos,school_name,height,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr),team_name,round,pick,year
0,Derek Abney,WR,Kentucky,69,179,4.58,33.5,,120,6.89,4.08,Baltimore Ravens / 7th / 244th pick / 2004,Baltimore Ravens,7,244,2004
1,Nathaniel Adibi,DE,Virginia Tech,75,254,4.65,33.0,,118,,4.36,Pittsburgh Steelers / 5th / 145th pick / 2004,Pittsburgh Steelers,5,145,2004
2,Roc Alexander,CB,Washington,71,185,4.35,39.5,,125,6.97,4.23,,,0,0,2004
3,Rich Alexis,RB,Washington,72,210,4.64,32.0,,114,7.41,4.43,,,0,0,2004
4,Jared Allen,DE,Idaho St.,78,265,4.72,33.0,13,120,7.11,4.34,Kansas City Chiefs / 4th / 126th pick / 2004,Kansas City Chiefs,4,126,2004


### Create Position Table

In [16]:
position_dim = agg_df[["Pos"]].drop_duplicates().reset_index(drop=True)
position_dim = position_dim.rename(columns={"Pos": "position_abrev"})
position_dim.head(2)

,position_abrev
0,WR
1,DE


In [17]:
# Create full position name column

pos_dict = {"WR": "Wide Receiver",
           "S": "Safety",
           "CB": "Cornerback",
           "TE": "Tight End",
           "RB": "Running Back",
           "C": "Center",
           "OLB": "Outside Linebacker",
           "ILB": "Inside Linebacker",
           "OG": "Offensive Guard",
           "QB": "Quarterback",
           "K": "Kicker",
           "OT": "Offensive Tackle",
           "DT": "Defensive Tackle",
           "P": "Punter",
           "DE": "Defensive End",
           "FB": "Fullback",
           "LS": "Long Snapper",
           "DL": "Defensive Line",
           "EDGE": "Edge Rusher",
           "LB": "Linebacker",
           "DB": "Defensive Back",
           "OL": "Offensive Line"}

In [18]:
# Create Offense/Defense/Special teams designation column

unit_dict = {"WR": "Offense",
           "S": "Defense",
           "CB": "Defense",
           "TE": "Offense",
           "RB": "Offense",
           "C": "Offense",
           "OLB": "Defense",
           "ILB": "Defense",
           "OG": "Offense",
           "QB": "Offense",
           "K": "Special Teams",
           "OT": "Offense",
           "DT": "Defense",
           "P": "Special Teams",
           "DE": "Defense",
           "FB": "Offense",
           "LS": "Special Teams",
           "DL": "Defense",
           "EDGE": "Defense",
           "LB": "Defense",
           "DB": "Defense",
           "OL": "Offense"}

In [19]:
position_dim["position_name"] = position_dim["position_abrev"].map(pos_dict)

position_dim["unit"] = position_dim["position_abrev"].map(unit_dict)

position_dim["position_id"] = position_dim.index
position_dim["position_id"] = position_dim["position_id"].astype(int)
position_dim = position_dim[["position_id", "position_name", "position_abrev", "unit"]]
position_dim.head(2)

,position_id,position_name,position_abrev,unit
0,0,Wide Receiver,WR,Offense
1,1,Defensive End,DE,Defense


### Create Team Table

In [20]:
team_dim = agg_df[["team_name"]].drop_duplicates().reset_index(drop=True)
team_dim.head(2)

,team_name
0,Baltimore Ravens
1,Pittsburgh Steelers


In [21]:
# Create Team abbreviation column

team_dict = {'Green Bay Packers': "GB",
 'Los Angeles Rams': "LAR",
 'Atlanta Falcons': "ATL",
 'New York Jets': "NYJ",
 'Pittsburgh Steelers': "PIT",
 'Los Angeles Chargers': "LAC",
 'Minnesota Vikings': "MIN",
 'Denver Broncos': "DEN",
 'Carolina Panthers': "CAR",
 'New York Giants': "NYG",
 'Cleveland Browns': "CLE",
 'Houston Texans': "HOU",
 'Cincinnati Bengals': "CIN",
 'San Francisco 49ers': "SF",
 'Jacksonville Jaguars': "JAX",
 'Washington Commanders': "WAS",
 'Seattle Seahawks': "SEA",
 'Baltimore Ravens': "BAL",
 'Arizona Cardinals': "AZ",
 'Buffalo Bills': "BUF",
 'Chicago Bears': "CHI",
 'Las Vegas Raiders': "LV",
 'New Orleans Saints': "NO",
 'Dallas Cowboys': "DAL",
 'New England Patriots': "NE",
 'Detroit Lions': "DET",
 'Tampa Bay Buccaneers': "TB",
 'Kansas City Chiefs': "KC",
 'Philadelphia Eagles': "PHI",
 'Miami Dolphins': "MIA",
 'Tennessee Titans': "TEN",
 'Indianapolis Colts': "IND"}

In [22]:
# Create team division column

division_dict = {'Green Bay Packers': "NFC North",
 'Los Angeles Rams': "NFC West",
 'Atlanta Falcons': "NFC South",
 'New York Jets': "AFC East",
 'Pittsburgh Steelers': "AFC North",
 'Los Angeles Chargers': "AFC West",
 'Minnesota Vikings': "NFC North",
 'Denver Broncos': "AFC West",
 'Carolina Panthers': "NFC South",
 'New York Giants': "NFC East",
 'Cleveland Browns': "AFC North",
 'Houston Texans': "AFC South",
 'Cincinnati Bengals': "AFC North",
 'San Francisco 49ers': "NFC West",
 'Jacksonville Jaguars': "AFC South",
 'Washington Commanders': "NFC East",
 'Seattle Seahawks': "NFC West",
 'Baltimore Ravens': "AFC North",
 'Arizona Cardinals': "NFC West",
 'Buffalo Bills': "AFC East",
 'Chicago Bears': "NFC North",
 'Las Vegas Raiders': "AFC West",
 'New Orleans Saints': "NFC South",
 'Dallas Cowboys': "NFC East",
 'New England Patriots': "AFC East",
 'Detroit Lions': "NFC North",
 'Tampa Bay Buccaneers': "NFC South",
 'Kansas City Chiefs': "AFC West",
 'Philadelphia Eagles': "NFC East",
 'Miami Dolphins': "AFC East",
 'Tennessee Titans': "AFC South",
 'Indianapolis Colts': "AFC South"}

In [23]:
team_dim["team_abrev"] = team_dim["team_name"].map(team_dict)
team_dim["division"] = team_dim["team_name"].map(division_dict)

team_dim = team_dim.sort_values(by="division").reset_index(drop=True)
team_dim["team_id"] = team_dim.index
team_dim["team_id"] = team_dim["team_id"].astype(int)
team_dim = team_dim[["team_id", "team_name", "team_abrev", "division"]]
team_dim.head()

,team_id,team_name,team_abrev,division
0,0,Miami Dolphins,MIA,AFC East
1,1,New England Patriots,NE,AFC East
2,2,New York Jets,NYJ,AFC East
3,3,Buffalo Bills,BUF,AFC East
4,4,Cleveland Browns,CLE,AFC North


### Create Draft Selection Table

In [24]:
draft_dim = agg_df[["Drafted (tm/rnd/yr)", "team_name", "round", "pick", "year"]].drop_duplicates()
draft_dim = draft_dim.sort_values(by=["year", "pick"]).reset_index(drop=True)

draft_dim["draft_id"] = draft_dim.index
draft_dim["draft_id"] = draft_dim["draft_id"].astype(int)

draft_dim["drafted"] = np.where(draft_dim['round'] > 0, True, False)
draft_dim = draft_dim.merge(team_dim, on="team_name")
draft_dim = draft_dim[["draft_id", "drafted", "team_id", "round", "pick", "year", "Drafted (tm/rnd/yr)"]] \
                        .sort_values(by="draft_id") \
                        .reset_index(drop=True)

print(len(draft_dim))
draft_dim.head()

4313


,draft_id,drafted,team_id,round,pick,year,Drafted (tm/rnd/yr)
0,0,False,32,0,0,2004,
1,1,True,13,1,1,2004,San Diego Chargers / 1st / 1st pick / 2004
2,2,True,12,1,2,2004,Oakland Raiders / 1st / 2nd pick / 2004
3,3,True,30,1,3,2004,Arizona Cardinals / 1st / 3rd pick / 2004
4,4,True,19,1,4,2004,New York Giants / 1st / 4th pick / 2004


In [25]:
draft_dim.dtypes

draft_id                int64
drafted                  bool
team_id                 int64
round                   int64
pick                    int64
year                   object
Drafted (tm/rnd/yr)    object
dtype: object

### Create School Table

In [26]:
school_dim = agg_df[["school_name"]]
school_dim = school_dim.drop_duplicates() \
                        .sort_values(by="school_name") \
                        .reset_index(drop=True)

school_dim["school_id"] = school_dim.index
school_dim["school_id"] = school_dim["school_id"].astype(int)
school_dim = school_dim[["school_id", "school_name"]]
school_dim.head(2)

,school_id,school_name
0,0,Abilene Christian
1,1,Air Force


### Create Player Fact Table

In [27]:
agg_df = agg_df.reset_index(drop=True)
agg_df["player_id"] = agg_df.index
agg_df["player_id"] = agg_df["player_id"].astype(int)

player_fact_table = agg_df.merge(position_dim, left_on="Pos", right_on="position_abrev") \
                            .merge(draft_dim, left_on="Drafted (tm/rnd/yr)", right_on="Drafted (tm/rnd/yr)") \
                            .merge(school_dim, left_on="school_name", right_on="school_name")
        
player_fact_table = player_fact_table[["player_id", "position_id", "draft_id", "team_id", "school_id", "Player", "height", "Wt", "40yd", 
                   "Vertical", "Bench", "Broad Jump", "3Cone", "Shuttle"]]

player_fact_table = player_fact_table.sort_values(by="player_id").reset_index(drop=True).fillna('')

player_fact_table["height"] = player_fact_table["height"].replace("",0).astype(int)
player_fact_table["Wt"] = player_fact_table["Wt"].replace("",0).astype(int)
player_fact_table["40yd"] = player_fact_table["40yd"].replace("",0).astype(float)
player_fact_table["Vertical"] = player_fact_table["Vertical"].replace("",0).astype(float)
player_fact_table["Bench"] = player_fact_table["Bench"].replace("",0).astype(int)
player_fact_table["Broad Jump"] = player_fact_table["Broad Jump"].replace("",0).astype(int)
player_fact_table["3Cone"] = player_fact_table["3Cone"].replace("",0).astype(float)
player_fact_table["Shuttle"] = player_fact_table["Shuttle"].replace("",0).astype(float)


In [28]:
col_name_dict = {"Player": "player_name",
                "Wt": "weight",
                "40yd": "40_yard_dash",
                "Vertical": "vertical_jump",
                "Bench": "bench_press",
                "Broad Jump": "broad_jump",
                "3Cone": "3_cone",
                "Shuttle": "short_shuttle"}

player_fact_table = player_fact_table.rename(columns= col_name_dict)
player_fact_table.head()

,player_id,position_id,draft_id,team_id,school_id,player_name,height,weight,40_yard_dash,vertical_jump,bench_press,broad_jump,3_cone,short_shuttle
0,0,0,210,6,113,Derek Abney,69,179,4.58,33.5,0,120,6.89,4.08
1,1,1,140,7,270,Nathaniel Adibi,75,254,4.65,33.0,0,118,0.00,4.36
2,2,2,4122,32,274,Roc Alexander,71,185,4.35,39.5,0,125,6.97,4.23
3,2,2,3704,32,274,Roc Alexander,71,185,4.35,39.5,0,125,6.97,4.23
4,2,2,3473,32,274,Roc Alexander,71,185,4.35,39.5,0,125,6.97,4.23


In [29]:
# Save tables locally
# path = "/Users/Andrew/Desktop/Projects/NFL_draft/"

#player_fact_table.to_csv(path + "clean_data/player_fact_table.csv", index=False)
#school_dim.to_csv(path + "clean_data/school_dim.csv", index=False)
#draft_dim.to_csv(path + "clean_data/draft_dim.csv", index=False)
#team_dim.to_csv(path + "clean_data/team_dim.csv", index=False)
#position_dim.to_csv(path + "clean_data/position_dim.csv", index=False)
